In [1]:
import pandas as pd
from datetime import date, timedelta
import numpy as np

from sklearn.preprocessing import LabelEncoder

import pycountry
import pycountry_convert as pc

In [2]:
raw_data_path = 'data/raw/'
output_path = 'data/preprocessed/'

# Load raw data from disk (ORG)

In [3]:
degrees = pd.read_csv(raw_data_path+'degrees.csv')
event_appearances = pd.read_csv(raw_data_path+'event_appearances.csv')
jobs = pd.read_csv(raw_data_path+'jobs.csv')
org_parents = pd.read_csv(raw_data_path+'org_parents.csv')
organizations = pd.read_csv(raw_data_path+'organizations.csv')
people = pd.read_csv(raw_data_path+'people.csv')

acquisitions = pd.read_csv(raw_data_path+'acquisitions.csv')
funding_rounds = pd.read_csv(raw_data_path+'funding_rounds.csv')
ipos = pd.read_csv(raw_data_path+'ipos.csv')
investors = pd.read_csv(raw_data_path+'investors.csv')
investments = pd.read_csv(raw_data_path+'investments.csv')


In [4]:
#helper functions
def convert_datetime(df):
    cols = ['created_at', 'updated_at', 'founded_on', 'last_funding_on', 'closed_on','started_on','completed_on','ended_on','PreSeries_announced_on','RoundA_announced_on','RoundB_announced_on','RoundC_announced_on','RoundD_announced_on','went_public_on']
    cols2 = ['age_closed','age_operating','PreSeries_from_founded','RoundA_from_founded', 'RoundA_from_PreSeries', 'RoundB_from_founded','RoundB_from_PreSeries','RoundB_from_RoundA','RoundC_from_founded','RoundC_from_PreSeries','RoundC_from_RoundA','RoundC_from_RoundB','RoundD_from_founded','RoundD_from_PreSeries','RoundD_from_RoundA','RoundD_from_RoundB','RoundD_from_RoundC']    
    for col, data in df.iteritems():
        df[col] = pd.to_datetime(df[col], errors = 'coerce') if col in cols else df[col] # for out-of-bound datetimes -> NaT
    for col, data in df.iteritems():
        df[col] = pd.to_timedelta(df[col], errors = 'coerce') if col in cols2 else df[col] # for out-of-bound datetimes -> NaT
    return df

# for part 1------------------------
# for getting type of organization -> company, investor, or school
def feature_to_list(x):
    x = x.strip().split(',')
    if 'company' not in x:
        x.append('c')
    if 'investor' not in x:
        x.append('i')
    if 'school' not in x:
        x.append('s')
    x.sort()
    for i in range(len(x)):
        if x[i] == 'c' or x[i] == 'i' or x[i] == 's':
            x[i] = None
    return x

today = date.today()
def get_age_operating(d):
    try:
        res = pd.to_timedelta(today - d.to_pydatetime().date())
    except:
        res = pd.Timedelta('292y')
    return res

# for part 4-----------------------
# split roles into separate columns
def split_categories(x):
    return x.strip().split(',')

def concat_to_str(x):
    str_ = ''
    for i in x:
        str_ += i.strip()
        str_ += ','
    return str_[:-1] if len(str_) else ''

def strip_uninterested_categories(x, categories_of_interest):
    if x is np.nan:
        return x
    category_list = []
    x = split_categories(x)
    for category in x:
        if category in categories_of_interest:
            category_list.append(category)
    return concat_to_str(category_list)

In [5]:
organizations = convert_datetime(organizations)
degrees = convert_datetime(degrees)
event_appearances = convert_datetime(event_appearances)
jobs = convert_datetime(jobs)
org_parents = convert_datetime(org_parents)
people = convert_datetime(people)
acquisitions = convert_datetime(acquisitions)
funding_rounds = convert_datetime(funding_rounds)
ipos = convert_datetime(ipos)
investors = convert_datetime(investors)
investments = convert_datetime(investments)

# Examine data

In [6]:
organizations.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230973 entries, 0 to 1230972
Data columns (total 41 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   uuid                         1230973 non-null  object        
 1   name                         1230959 non-null  object        
 2   type                         1230973 non-null  object        
 3   permalink                    1230966 non-null  object        
 4   cb_url                       1230967 non-null  object        
 5   rank                         1230798 non-null  float64       
 6   created_at                   1230973 non-null  datetime64[ns]
 7   updated_at                   1230973 non-null  datetime64[ns]
 8   legal_name                   187626 non-null   object        
 9   roles                        1221975 non-null  object        
 10  domain                       1151205 non-null  object        
 11  homepage_ur

In [7]:
degrees.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369990 entries, 0 to 369989
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   uuid              369990 non-null  object        
 1   name              339822 non-null  object        
 2   type              369990 non-null  object        
 3   permalink         0 non-null       float64       
 4   cb_url            0 non-null       float64       
 5   rank              0 non-null       float64       
 6   created_at        369990 non-null  datetime64[ns]
 7   updated_at        369990 non-null  datetime64[ns]
 8   person_uuid       369990 non-null  object        
 9   person_name       369990 non-null  object        
 10  institution_uuid  369990 non-null  object        
 11  institution_name  369990 non-null  object        
 12  degree_type       358151 non-null  object        
 13  subject           342822 non-null  object        
 14  star

In [8]:
event_appearances.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413422 entries, 0 to 413421
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   uuid               413422 non-null  object        
 1   name               413422 non-null  object        
 2   type               413422 non-null  object        
 3   permalink          413422 non-null  object        
 4   cb_url             413422 non-null  object        
 5   rank               0 non-null       float64       
 6   created_at         413422 non-null  datetime64[ns]
 7   updated_at         413422 non-null  datetime64[ns]
 8   event_uuid         413422 non-null  object        
 9   event_name         413422 non-null  object        
 10  participant_uuid   413422 non-null  object        
 11  participant_name   413422 non-null  object        
 12  participant_type   413422 non-null  object        
 13  appearance_type    413422 non-null  object  

In [9]:
jobs.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589222 entries, 0 to 1589221
Data columns (total 17 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   uuid         1589222 non-null  object        
 1   name         1589222 non-null  object        
 2   type         1589222 non-null  object        
 3   permalink    1589220 non-null  object        
 4   cb_url       1589220 non-null  object        
 5   rank         0 non-null        float64       
 6   created_at   1589222 non-null  datetime64[ns]
 7   updated_at   1589222 non-null  datetime64[ns]
 8   person_uuid  1589222 non-null  object        
 9   person_name  1589222 non-null  object        
 10  org_uuid     1589222 non-null  object        
 11  org_name     1589218 non-null  object        
 12  started_on   786363 non-null   datetime64[ns]
 13  ended_on     293929 non-null   datetime64[ns]
 14  is_current   1589222 non-null  bool          
 15  title        15

In [10]:
org_parents.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16982 entries, 0 to 16981
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uuid         16982 non-null  object        
 1   name         16982 non-null  object        
 2   type         16982 non-null  object        
 3   permalink    16982 non-null  object        
 4   cb_url       16982 non-null  object        
 5   rank         16982 non-null  int64         
 6   created_at   16982 non-null  datetime64[ns]
 7   updated_at   16982 non-null  datetime64[ns]
 8   parent_uuid  16982 non-null  object        
 9   parent_name  16982 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 1.3+ MB


In [11]:
people.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106474 entries, 0 to 1106473
Data columns (total 22 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   uuid                            1106474 non-null  object        
 1   name                            1106474 non-null  object        
 2   type                            1106474 non-null  object        
 3   permalink                       1106473 non-null  object        
 4   cb_url                          1106473 non-null  object        
 5   rank                            1104160 non-null  float64       
 6   created_at                      1106474 non-null  datetime64[ns]
 7   updated_at                      1106474 non-null  datetime64[ns]
 8   first_name                      1106443 non-null  object        
 9   last_name                       1106444 non-null  object        
 10  gender                          1084964 no

In [12]:
acquisitions.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109825 entries, 0 to 109824
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   uuid                   109825 non-null  object        
 1   name                   109825 non-null  object        
 2   type                   109825 non-null  object        
 3   permalink              109825 non-null  object        
 4   cb_url                 109825 non-null  object        
 5   rank                   109824 non-null  float64       
 6   created_at             109825 non-null  datetime64[ns]
 7   updated_at             109825 non-null  datetime64[ns]
 8   acquiree_uuid          109825 non-null  object        
 9   acquiree_name          109825 non-null  object        
 10  acquiree_cb_url        109825 non-null  object        
 11  acquiree_country_code  103573 non-null  object        
 12  acquiree_state_code    56940 non-null   obje

In [13]:
funding_rounds.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366609 entries, 0 to 366608
Data columns (total 24 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   uuid                                366609 non-null  object        
 1   name                                366609 non-null  object        
 2   type                                366609 non-null  object        
 3   permalink                           366608 non-null  object        
 4   cb_url                              366608 non-null  object        
 5   rank                                366562 non-null  float64       
 6   created_at                          366609 non-null  datetime64[ns]
 7   updated_at                          366609 non-null  datetime64[ns]
 8   country_code                        362335 non-null  object        
 9   state_code                          192266 non-null  object        
 10  region  

In [14]:
ipos.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35244 entries, 0 to 35243
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   uuid                           35244 non-null  object        
 1   name                           0 non-null      float64       
 2   type                           35244 non-null  object        
 3   permalink                      35244 non-null  object        
 4   cb_url                         35244 non-null  object        
 5   rank                           35239 non-null  float64       
 6   created_at                     35244 non-null  datetime64[ns]
 7   updated_at                     35244 non-null  datetime64[ns]
 8   org_uuid                       35244 non-null  object        
 9   org_name                       35244 non-null  object        
 10  org_cb_url                     35244 non-null  object        
 11  country_code   

In [15]:
investors.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168191 entries, 0 to 168190
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   uuid                         168191 non-null  object        
 1   name                         168189 non-null  object        
 2   type                         168191 non-null  object        
 3   permalink                    168191 non-null  object        
 4   cb_url                       168191 non-null  object        
 5   rank                         167816 non-null  float64       
 6   created_at                   168191 non-null  datetime64[ns]
 7   updated_at                   168191 non-null  datetime64[ns]
 8   roles                        168191 non-null  object        
 9   domain                       66476 non-null   object        
 10  country_code                 113599 non-null  object        
 11  state_code                

In [16]:
investments.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552420 entries, 0 to 552419
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   uuid                552420 non-null  object        
 1   name                552417 non-null  object        
 2   type                552420 non-null  object        
 3   permalink           552416 non-null  object        
 4   cb_url              552416 non-null  object        
 5   rank                0 non-null       float64       
 6   created_at          552420 non-null  datetime64[ns]
 7   updated_at          552420 non-null  datetime64[ns]
 8   funding_round_uuid  552419 non-null  object        
 9   funding_round_name  552417 non-null  object        
 10  investor_uuid       552420 non-null  object        
 11  investor_name       552419 non-null  object        
 12  investor_type       552420 non-null  object        
 13  is_lead_investor    266915 no

# Part1 - Preprocessing organizational and managerial features

### 1. organizations

In [17]:
distil_organizations = organizations.drop([
    'created_at',
    'updated_at',
#     'rank', # need for extracting schools' rank, drop later
    'permalink',
    'legal_name',
#     'cb_url', # for front-end dataset
    'address',
    'phone',
#     'homepage_url', # for front-end dataset
    'logo_url',
    'postal_code',
    'region',
    'city',
    'short_description',
    'alias1', 
    'alias2',
    'alias3',
    'total_funding',# total_funding_usd is used
    'total_funding_currency_code',
    'email',
    'type', # all = 'organization'
], axis=1)

In [18]:
# dropping
organizations_drop = distil_organizations.copy()

# remove rows without company name (checked, they don't have alias either)
organizations_drop = organizations_drop[organizations_drop['name'].notnull()]

# remove illogical instances with founded_on < closed_on
organizations_drop = organizations_drop[~(organizations_drop['closed_on'] < organizations_drop['founded_on'])]
organizations_drop = organizations_drop.reset_index(drop=True)

In [19]:
# filling
organizations_fill = organizations_drop.copy()

# fill null in 'num_exits' as 0
organizations_fill['num_exits'] = organizations_fill['num_exits'].fillna(0).astype(int)

# if 'roles' is empty, get value from 'primary_role'
organizations_fill['roles'] = organizations_fill['roles'].fillna(organizations_fill['primary_role'])

# if missing country code or state code, fill with "Unknown"
organizations_fill['country_code'] = organizations_fill['country_code'].fillna('Unknown')
organizations_fill['state_code'] = organizations_fill['state_code'].fillna('Unknown')

In [20]:
# transforming
organizations_transform = organizations_fill.copy()

# change to 1 flag: has_domain, this signifies whether the firm is well-developed/ has influence
organizations_transform['has_domain'] = organizations_transform['domain'].notnull()

# split roles into separate columns
organizations_transform['roles'] = organizations_transform['roles'].apply(func = feature_to_list)
organizations_transform[['role1','role2','role3']] = pd.DataFrame(organizations_transform.roles.tolist())
organizations_transform = organizations_transform.reset_index(drop=True)

# change 'roles' to 3 flags: is_company, is_investor, is_school,is_primary_company, is_primary_investor, is_primary_school
roles_dummy = pd.get_dummies(organizations_transform[['role1','role2','role3']], prefix = 'is')
primary_role_dummy = pd.get_dummies(organizations_transform['primary_role'], prefix = 'is_primary')
dummy = pd.concat((roles_dummy, primary_role_dummy), axis = 1) 
organizations_transform = pd.concat((organizations_transform, dummy), axis = 1) # add 6 columns to main org dataframe
organizations_transform = organizations_transform.reset_index(drop=True)

# change to 4 flags: is_acquired, is_ipo, is_operating, is_closed
status_dummy = pd.get_dummies(organizations_transform['status'], prefix = 'is') # 4 one-hot encoded columns
organizations_transform = pd.concat((organizations_transform, status_dummy), axis = 1) # add 4 columns to main org dataframe
organizations_transform = organizations_transform.reset_index(drop=True)
organizations_transform['is_operating'] = organizations_transform.where(organizations_transform['is_closed']==1, 1)['is_operating']

# create age_closed (for company that closed), age_operating (how old the firm is today) for every company in days
age_closed = organizations_transform[organizations_transform['closed_on'].notnull()]['closed_on'] - organizations_transform[organizations_transform['closed_on'].notnull()]['founded_on']
age_closed = pd.DataFrame({'age_closed': age_closed})
organizations_transform = pd.concat((organizations_transform, age_closed), axis = 1) # add age_closed to main org dataframe
organizations_transform = organizations_transform.reset_index(drop=True)

age_operating = organizations_transform[organizations_transform['closed_on'].isnull() & organizations_transform['founded_on'].notnull()]['founded_on']
age_operating = age_operating.apply(func=get_age_operating)
age_operating = pd.DataFrame({'age_operating': age_operating})
organizations_transform = pd.concat((organizations_transform, age_operating), axis = 1) # add age_operating to main org dataframe
organizations_transform = organizations_transform.reset_index(drop=True)

# convert facebook/linkedin/twitter urls to number_of_social_media_url, it signifies how much social influence the firm has
organizations_transform['has_facebook'] = organizations_transform['facebook_url'].notnull().astype(int)
organizations_transform['has_linkedin'] = organizations_transform['linkedin_url'].notnull().astype(int)
organizations_transform['has_twitter'] = organizations_transform['twitter_url'].notnull().astype(int)
organizations_transform['num_social_media_url_org'] = organizations_transform['has_facebook']+organizations_transform['has_linkedin']+organizations_transform['has_twitter']

# convert founded_on and closed_on from timestamp to year
organizations_transform['founded_on_year'] = organizations_transform['founded_on'].dt.year
organizations_transform['closed_on_year'] = organizations_transform['closed_on'].dt.year

C:\Users\emily\Anaconda3\envs\TEMG4952A\lib\site-packages\pandas\core\series.py:4108: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [21]:
# prepare a list of top_500_schools, can be later used for identifying educational backgrounds of founders/emplyees
# schools are ranked based on crunchbase's given rank
top_500_schools = organizations_transform[organizations_transform['is_school']==1].sort_values(by=['rank']).head(500)

In [22]:
organizations_transform = organizations_transform.drop([
    'roles',
    'role1',
    'role2',
    'role3',
    'primary_role',
    'status',
#     'domain', # for front-end usage
#     'facebook_url', 
#     'linkedin_url',
#     'twitter_url',
    'rank'
], axis=1)

In [23]:
# for checking
# organizations_transform.info()
# organizations_transform.head(10)
# organizations_transform.to_csv('organizations_transform.csv')

### 2. degrees

In [24]:
distil_degrees = degrees.drop([
    'permalink',
    'cb_url',
    'rank', # all null
    'type', # all == 'degree' 
    'name', # name is basically institution name + subject
    'created_at',
    'updated_at',
    'started_on', # not meaningful, and half of them are null
    'completed_on' # not meaningful, and half of them are null
],axis=1)

In [25]:
# dropping
degrees_drop = distil_degrees.copy()

# remove rows without subject name because they are not credible
degrees_drop = degrees_drop[degrees_drop['subject'].notnull()] 

# too costly to process(deriving degree_type), remove instances without degree_type
degrees_drop = degrees_drop[degrees_drop['degree_type'].notnull()] 

In [26]:
# filling
degrees_fill = degrees_drop.copy()

In [27]:
# transforming
degrees_transform = degrees_fill.copy()

# add is_from_top_500_schools to identify 'good' educational background. uses rank prepared from organizations
top_500_schools = pd.DataFrame({'institution_uuid': top_500_schools['uuid'], 'is_from_top_500_schools': 1})
degrees_transform = degrees_transform.merge(top_500_schools, how='left', on='institution_uuid') 
degrees_transform['is_from_top_500_schools'] = degrees_transform['is_from_top_500_schools'].fillna(0) # 0 means the degree is not from top schools

In [28]:
# for checking
# degrees_transform.info()
# degrees_transform.head(10)
# degrees_transform.to_csv('degrees_transform.csv')

### 3. event_appearances

In [29]:
distil_event_appearances = event_appearances.drop([
    'permalink', # not useful
    'cb_url', # not useful
    'short_description', # wedon't do sentiment analysis
    'rank', # all null
    'created_at', # not useful
    'updated_at', # not useful
    'type', # not meaningful, they are all 'event_appearance'
    'event_name', # not useful, we only count exposure by number
    'name' # basically event name + organizer
],axis=1)

In [30]:
# for checking:
# no processing needed in this stage. We only count the number of appearances
event_appearances_transform = distil_event_appearances.copy()
# event_appearances_transform.info()
# event_appearances_transform.head(10)
# event_appearances_transform.to_csv('event_appearances_transform.csv')

### 4. jobs

In [31]:
distil_jobs = jobs.drop([
    'permalink', # not useful
    'cb_url', # not useful
    'rank', # all null
    'org_name', # there are null values, and we use uuid to make reference anyways
    'created_at', # not useful
    'updated_at', # not useful
    'type', # not meaningful, they are all 'job'
    'name', # full name only
],axis=1)

In [32]:
# dropping
jobs_drop = distil_jobs.copy()

# according to job_type, the thrown ones are mostly employees, insignificant to our target(founder, ceo ...)
# we are concern about founders' jobs & experiences
jobs_drop = jobs_drop[jobs_drop['title'].notnull()]

# remove illogical instances with started_on < ended_on
jobs_drop = jobs_drop[~(jobs_drop['started_on'] > jobs_drop['ended_on'])]
jobs_drop = jobs_drop.reset_index(drop=True)

In [33]:
# filling
jobs_fill = jobs_drop.copy()

In [34]:
# transforming
jobs_transform = jobs_fill.copy()

#job types to boolean flags : is_executive, is_employee, etc.
job_type_dummy = pd.get_dummies(jobs_transform['job_type'], prefix = 'is')
jobs_transform = pd.concat((jobs_transform, job_type_dummy), axis = 1)

#get job duration for both ended jobs and current job (in days)
duration_ended = jobs_transform[jobs_transform['ended_on'].notnull()]['ended_on'] - jobs_transform[jobs_transform['ended_on'].notnull()]['started_on']
duration_ended = pd.DataFrame({'duration_ended': duration_ended})
jobs_transform = pd.concat((jobs_transform, duration_ended), axis = 1)
jobs_transform = jobs_transform.reset_index(drop=True)

duration_current = jobs_transform[jobs_transform['ended_on'].isnull() & jobs_transform['started_on'].notnull()]['started_on']
duration_current = duration_current.apply(func=get_age_operating) # same method with getting company's age earlier
duration_current = pd.DataFrame({'duration_current': duration_current})
jobs_transform = pd.concat((jobs_transform, duration_current), axis = 1)
jobs_transform = jobs_transform.reset_index(drop=True)

# is_founder, founder is not a type of job but contain in job title. All jobs with string 'founder', 'co-founder' etc will have value of 1 in is_founder
jobs_transform['is_founder'] = jobs_transform['title'].where(jobs_transform['title'].str.contains("founder|Founder")).notnull()

In [35]:
# for checking:
# jobs_transform.info()
# jobs_transform.head(10)
# jobs_transform.to_csv('jobs_transform.csv')

### 5. org_parents

In [36]:
distil_org_parents = org_parents.drop([
    'permalink',
    'cb_url', # not useful
    'rank', # available in distil_organizations
    'type', # not meaningful, they all are 'organization'
    'created_at', # not useful
    'updated_at' # not useful
],axis=1)

In [37]:
# for checking:
# no processing needed in this stage. 
org_parents_transform = distil_org_parents.copy()
# org_parents_transform.info()
# org_parents_transform.head(10)
# org_parents_transform.to_csv('org_parents_transform.csv')

### 6. people

In [38]:
distil_people = people.drop([
    'permalink', # not useful
    'cb_url',# not useful
    'logo_url',# not useful
    'type', # not useful, they are all 'person'
    'created_at',# not useful
    'updated_at', # not useful
    'region', # too much granularity
    'city', # too much granularity, we already have country & state code
    'rank', # not useful
    'name', # not useful, basically first name + last name
    'featured_job_organization_name' # we use uuid to make reference anyways
], axis=1)

In [39]:
# dropping
people_drop = distil_people.copy()

# drop insignificant instances without name
people_drop = people_drop[people_drop['first_name'].notnull()]
people_drop = people_drop[people_drop['last_name'].notnull()]
people_drop = people_drop.reset_index(drop=True)

In [40]:
# filling
people_fill = people_drop.copy()

#create new category for unknowns, easier management
people_fill['country_code'] = people_fill['country_code'].fillna('Unknown')
people_fill['state_code'] = people_fill['state_code'].fillna('Unknown')

In [41]:
# transforming
people_transform = people_fill.copy()

# like organition, social media convert to number_of_social_media_url
people_transform['has_facebook'] = people_transform['facebook_url'].notnull().astype(int)
people_transform['has_linkedin'] = people_transform['linkedin_url'].notnull().astype(int)
people_transform['has_twitter'] = people_transform['twitter_url'].notnull().astype(int)
people_transform['num_social_media_url'] = people_transform['has_facebook']+people_transform['has_linkedin']+people_transform['has_twitter']

# convert gender to is_male and is_female
# there are too many entries with gender neither F/M, too costly to process so they are omitted.
genders = pd.get_dummies(people_transform['gender'], prefix='is')
people_transform[['is_male','is_female']] = genders[['is_male','is_female']]

In [42]:
people_transform = people_transform.drop([
    'facebook_url',
    'linkedin_url',
    'twitter_url',
    'gender' # converted to boolean flags
], axis=1)

In [43]:
#for checking:
# people_transform.info()
# people_transform.head(10)
# people_transform.to_csv('people_transform.csv')

# Part 1 - Concatenating org & managerial features

### organizations
- org_parents_transform -> add parent_uuid, parent_name
- event_appearances -> add num_event_appearances_org
- jobs -> num_total_jobs, num_current_jobs

In [44]:
organizations_concat = organizations_transform.copy()

In [45]:
# concatenating org_parents

#-> add parent to organization, including parent name and parent uuid, and a boolean flag to signify if it has a parent
organizations_concat = organizations_concat.merge(org_parents_transform.drop(['name'],axis=1),how='left', on='uuid')
organizations_concat['has_parent'] = organizations_concat['parent_uuid'].notnull()

In [46]:
# concatenating event_appearances

# count num_event_appearances_org in all time
temp = event_appearances_transform[event_appearances_transform['participant_type']=='organization'].copy()
temp['num_event_appearances_org'] = 1
num_event_appearances_org = temp.groupby(['participant_uuid']).sum() # a dataframe with parti_uuid as unique entries and a column of frequency
num_event_appearances_org.index.name = 'uuid'
organizations_concat = organizations_concat.merge(num_event_appearances_org,how='left', on='uuid') # add the column to main org dataframe
organizations_concat['num_event_appearances_org'] = organizations_concat['num_event_appearances_org'].fillna(0) # if null meaning no event_appearances available = 0 times

In [49]:
# concatenating jobs

# num_total_jobs: historically how many jobs of a company has been posted on crunchbase
num_total_jobs = pd.DataFrame(jobs_transform['org_uuid'].value_counts()) # counting how many times does a org's uuid appear = how many jobs it has
num_total_jobs = num_total_jobs.rename(columns={"org_uuid":"num_total_jobs"})
num_total_jobs.index.name = 'uuid'

organizations_concat = organizations_concat.merge(num_total_jobs,how='left', on='uuid')
organizations_concat['num_total_jobs'] = organizations_concat['num_total_jobs'].fillna(0)

# num_current_jobs： historically how many jobs of a company has been posted on crunchbase and they are current
num_current_jobs = pd.DataFrame(jobs_transform[jobs_transform['is_current']==1]['org_uuid'].value_counts())
num_current_jobs = num_current_jobs.rename(columns={"org_uuid":"num_current_jobs"})
num_current_jobs.index.name = 'uuid'

organizations_concat = organizations_concat.merge(num_current_jobs,how='left', on='uuid') # add the 2 columns to main org dataframe
organizations_concat['num_current_jobs'] = organizations_concat['num_current_jobs'].fillna(0) # if missing = 0 jobs

In [50]:
# organizations_concat.head()
# organizations_concat.to_csv('organizations_concat.csv')

### people 
- event_appearances -> add num_event_appearances_person
- degrees -> add num_completed_degrees, num_incomplete_degrees, num_completed_degrees_from_top_500_schools, num_incomplete_degrees_from_top_500_schools

In [51]:
people_concat = people_transform.copy()

In [52]:
# concatenating event_appearances

# count num_event_appearances_person
num_event_appearances_person = pd.DataFrame(event_appearances_transform[event_appearances_transform['participant_type']=='person']['participant_uuid'].value_counts()) # counting how many times does a person's uuid appear = how many times he appeared
num_event_appearances_person = num_event_appearances_person.rename(columns={"participant_uuid":"num_event_appearances_person"})
num_event_appearances_person.index.name = 'uuid'
people_concat = people_concat.merge(num_event_appearances_person,how='left', on='uuid') # add the column to people dataframe
people_concat['num_event_appearances_person'] = people_concat['num_event_appearances_person'].fillna(0) # if null meaning no event_appearances available = 0 times

In [53]:
# concatenating degrees

# num_completed_degrees
num_completed_degrees = pd.DataFrame(degrees_transform[degrees_transform['is_completed']==1]['person_uuid'].value_counts()) # counting how many times does a person's uuid appear = how many degrees he holds 
num_completed_degrees.index.name = 'uuid'
people_concat = people_concat.merge(num_completed_degrees.rename(columns={"person_uuid":"num_completed_degrees"}),how='left', on='uuid') # add the column to people dataframe
people_concat['num_completed_degrees'] = people_concat['num_completed_degrees'].fillna(0)

# num_incomplete_degrees
num_incomplete_degrees = pd.DataFrame(degrees_transform[degrees_transform['is_completed']==0]['person_uuid'].value_counts()) # same logic
num_incomplete_degrees.index.name = 'uuid'
people_concat = people_concat.merge(num_incomplete_degrees.rename(columns={"person_uuid":"num_incomplete_degrees"}),how='left', on='uuid') # add the column to people dataframe
people_concat['num_incomplete_degrees'] = people_concat['num_incomplete_degrees'].fillna(0)

# num_completed_degrees_from_top_500_schools
completed_degrees = degrees_transform[degrees_transform['is_completed']==1]
num_completed_degrees_from_top_500_schools = pd.DataFrame(completed_degrees[completed_degrees['is_from_top_500_schools']==1]['person_uuid'].value_counts()) # same logic
num_completed_degrees_from_top_500_schools.index.name = 'uuid'
people_concat = people_concat.merge(num_completed_degrees_from_top_500_schools.rename(columns={"person_uuid":"num_completed_degrees_from_top_500_schools"}),how='left', on='uuid') # add the column to people dataframe
people_concat['num_completed_degrees_from_top_500_schools'] = people_concat['num_completed_degrees_from_top_500_schools'].fillna(0)

# num_incomplete_degrees_from_top_500_schools
incomplete_degrees = degrees_transform[degrees_transform['is_completed']==0]
num_incomplete_degrees_from_top_500_schools = pd.DataFrame(incomplete_degrees[incomplete_degrees['is_from_top_500_schools']==1]['person_uuid'].value_counts()) # same logic
num_incomplete_degrees_from_top_500_schools.index.name = 'uuid'
people_concat = people_concat.merge(num_incomplete_degrees_from_top_500_schools.rename(columns={"person_uuid":"num_incomplete_degrees_from_top_500_schools"}),how='left', on='uuid') # add the column to people dataframe
people_concat['num_incomplete_degrees_from_top_500_schools'] = people_concat['num_incomplete_degrees_from_top_500_schools'].fillna(0)

In [54]:
# people_concat.head()
# people_concat.to_csv('people_concat.csv')

### jobs
- org -> FAANG_experience
- people -> link all features here so it's easier to evaluate a firm's cumulative employee background by entries in jobs

In [55]:
jobs_concat = jobs_transform.copy()

In [56]:
# concatenating people

# taking the meaningful & transformed columns in people_concat
jobs_concat = jobs_concat.merge(people_concat[['has_facebook',
                                               'has_linkedin',
                                               'has_twitter',
                                               'uuid',
                                               'num_social_media_url',
                                               'num_event_appearances_person',
                                               'num_completed_degrees', 
                                               'num_incomplete_degrees',
                                               'num_completed_degrees_from_top_500_schools',
                                                'num_incomplete_degrees_from_top_500_schools',
                                               'is_male',
                                               'is_female']].rename(columns={'uuid':'person_uuid'}),on='person_uuid')

# identify male and female founders
jobs_concat['is_male_founder']=jobs_concat['is_male']&jobs_concat['is_founder']
jobs_concat['is_female_founder']=jobs_concat['is_female']&jobs_concat['is_founder']

In [57]:
# creating has_FAANG_exp

'''
df662812-7f97-0b43-9d3e-12f64f504fbb   Facebook (unique in organization)
7063d087-96b8-2cc1-ee88-c221288acc2a   Apple (unique in organization)
3a7ec450-5422-1553-6c6a-4b28f6d4a17c   Netflix (unique in organization)

Amazon: 46 instances in org with employees>10
organizations_transform[organizations_transform['name'].str.contains('Amazon')].sort_values('employee_count')[['employee_count','is_operating','name']][organizations_transform['employee_count']!=0]

Google: 19 instances in org with employees>10
organizations_transform[organizations_transform['name'].str.contains('Google')][organizations_transform['employee_count']!=0]

'''
# create is_FAANG for jobs
FAANG_uuid = organizations_transform[organizations_transform['name'].str.contains('Google')][organizations_transform['employee_count']!=0]['uuid'] # extracting Google's uuids
FAANG_uuid = FAANG_uuid.append(organizations_transform[organizations_transform['name'].str.contains('Amazon')][organizations_transform['employee_count']!=0]['uuid']) # extracting Amazon's uuids
FAANG_uuid = FAANG_uuid.append(pd.Series(['df662812-7f97-0b43-9d3e-12f64f504fbb','7063d087-96b8-2cc1-ee88-c221288acc2a','3a7ec450-5422-1553-6c6a-4b28f6d4a17c'])) # appeding FB, Apple, Netflix uuids
FAANG_uuid = pd.DataFrame({'is_FAANG':1, 'org_uuid':FAANG_uuid})

jobs_concat = jobs_concat.merge(FAANG_uuid, how='left', on='org_uuid') # link value is_FAANG = 1 for those jobs with organizations in FAANG. 
jobs_concat['is_FAANG'] = jobs_concat['is_FAANG'].fillna(0) # if no '1', then it is not FAANG -> 0

# create has_FAANG_exp_person for people
has_FAANG_exp_person = jobs_concat[['person_uuid','is_FAANG']].groupby(['person_uuid']).sum() # identify what people has FAANG exp and the number stands for # of FAANG jobs
has_FAANG_exp_person = has_FAANG_exp_person.rename(columns={'is_FAANG': 'has_FAANG_exp_person'})

# create has_FAANG_exp_person for jobs
jobs_concat = jobs_concat.merge(has_FAANG_exp_person, how='left', on='person_uuid') # link person info to jobs

# create has_FAANG_exp_founder for organizations
jobs_concat['has_FAANG_exp_founder'] = jobs_concat['is_founder'] & jobs_concat['has_FAANG_exp_person'] # for job entries that are 'founder' and the person has_FAANG_exp, we create a boolean flag for them.

C:\Users\emily\Anaconda3\envs\TEMG4952A\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
C:\Users\emily\Anaconda3\envs\TEMG4952A\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [58]:
# jobs_concat.info()
# jobs_concat.to_csv('jobs_concat.csv')

### people + organizations + jobs

In [59]:
# group jobs_concat by org, we are essentially evaluating the cumulative features of each company.
temp = jobs_concat.groupby(['org_uuid']).sum()

In [60]:
# take the useful features only and rename them since they are 'sums' now and are feature or the orgamization.
temp = pd.DataFrame({'num_male_founder':temp['is_male_founder'],
                     'num_female_founder':temp['is_female_founder'],
                     'num_event_appearances_employee':temp['num_event_appearances_person'],
                     'num_completed_degrees_employee':temp['num_completed_degrees'],
                     'num_incomplete_degrees_employee':temp['num_incomplete_degrees'],
                     'num_completed_degrees_from_top_500_schools':temp['num_completed_degrees_from_top_500_schools'],
                     'num_incomplete_degrees_from_top_500_schools':temp['num_incomplete_degrees_from_top_500_schools'],
                     'num_FAANG_exp_founder':temp['has_FAANG_exp_founder'],
                    'percentage_of_male_founder': temp['is_male_founder'] / (temp['is_male_founder']+temp['is_female_founder']), # calculate % of male & female founders, so it doesnt get affected by size of company
                    'percentage_of_female_founder': temp['is_female_founder'] / (temp['is_male_founder']+temp['is_female_founder']),
                      })
temp.index.name = 'uuid'

In [61]:
organizations_p1 = organizations_concat.merge(temp, on='uuid') # put the newly dericed features related to founders background, managerial features to the main org dataframe

In [62]:
# organizations_p1.info()
# organizations_p1.to_csv('organizations_concat_p1.csv')

# Part2 Preprocessing: funding rounds related features

### 7. acquisitions

In [63]:
distil_acquisitions = acquisitions.drop([
    'type',
    'created_at', # not useful
    'updated_at',
    'permalink',
    'cb_url',
    'acquiree_cb_url',
    'acquirer_cb_url',
    'acquiree_region', # too detailed---
    'acquiree_city',
    'price',
    'price_currency_code',
    'rank',
    'acquirer_region',
    'acquirer_city',
    'acquisition_type',
    'uuid',  # since uuid of the transaction
    'acquiree_name', # extra feature during cocatenation of dataframes
    'name', # irrelevant during concatenation,
    'acquiree_country_code', # extra during concatenation
    'acquiree_state_code' # extra during concatenation
],axis=1)

In [64]:
# filling & transform
acquisitions_transform = distil_acquisitions.copy()

#missing values for state_code's and country_code's, price_usd--> not initializing it to 0 --> not needed for acquiree after concatenation
acquisitions_transform['acquirer_country_code']= acquisitions_transform['acquirer_country_code'].fillna(0) #if country_code=NaN, then just fill 0
acquisitions_transform['acquirer_state_code']=acquisitions_transform['acquirer_state_code'].fillna(0) #if state_code= NaN, then just fill 0

#create BOOLEAN FLAG for is_acquisition_price
acquisitions_transform['is_acquisition_price']= np.where(acquisitions_transform['price_usd'].notnull(),1,0) # 1 for share price, 0 for no

In [65]:
# acquisitions_transform.info(verbose=True)
# acquisitions_transform.head(10)

### 8. funding_rounds

In [66]:
distil_funding_rounds = funding_rounds.drop([
    'created_at',
    'updated_at',
    'permalink',
    'cb_url',
    'state_code',
    'region',
    'city',
    'post_money_valuation', # we use post_money_valuation_usd
    'post_money_valuation_currency_code',
    'rank',
    'raised_amount',
    'raised_amount_currency_code',
    'type', # only one type='funding_rounds',
    'name', # name of investment is irrelevant
    'org_name',
    'country_code', # duplicate during concatenation
    'lead_investor_uuids' # irrelevant
],axis=1)

In [67]:
# filling & transform
funding_rounds_transform = distil_funding_rounds.copy()

# drop the rows with no raised_amount_usd (366,609 instances originally -> 267271 afterwards)
funding_rounds_transform.drop(funding_rounds_transform[funding_rounds_transform['raised_amount_usd'].isnull()].index, inplace=True) 

# creating BOOLEAN FLAGS to identify which stage of the funding round belongs to
# 1) is_PreSeries= seed, grant, angel, pre-seed, equity_crowdfunding, private_equity, undisclosed, corporate_round, product_crowdfunding, non_equity_assistance
funding_rounds_transform['is_PreSeries']= np.where(funding_rounds_transform['investment_type'].str.contains('seed|grant|angel|pre_seed|equity_crowdfunding|private_equity|undisclosed|corporate_round|product_crowdfunding|non_equity_assistance'),1,0) # 1 for true, 0 for false

# 2) is_RoundA = series_a
funding_rounds_transform['is_RoundA']= np.where(funding_rounds_transform['investment_type'].str.contains('series_a'),1,0)

# 3) is_RoundB= series_b
funding_rounds_transform['is_RoundB']= np.where(funding_rounds_transform['investment_type'].str.contains('series_b'),1,0)

# 4) is_RoundC= series_c
funding_rounds_transform['is_RoundC']= np.where(funding_rounds_transform['investment_type'].str.contains('series_c'),1,0)

#5) is_RoundD= series_d, series_unknown, convertible_note, series_e, series_f, series_g, series_h, series_i, series_j, 
funding_rounds_transform['is_RoundD']= np.where(funding_rounds_transform['investment_type'].str.contains('series_d|series_unknown|convertible_note|series_e|series_f|series_g|series_h|series_i|series_j'),1,0)                                                                  

In [68]:
# funding_rounds_transform.info(verbose=True)
# funding_rounds_transform.head(8)

### 9. ipos

In [69]:
distil_ipos = ipos.drop([
    'created_at',
    'updated_at',
    'state_code',
    'region',
    'city',
    'stock_exchange_symbol',
    'share_price',
    'share_price_currency_code',
    'permalink',
    'cb_url',
    'org_cb_url',
    'type',
    'valuation_price',
    'valuation_price_currency_code',
    'money_raised',
    'money_raised_currency_code',
    'rank',
    'name',
    'uuid' #ipo uuid not relevant
],axis=1)

In [70]:
ipos_transform = distil_ipos.copy()

### 10. investors

In [71]:
distil_investors =investors.drop([
    'created_at',
    'updated_at',
    'permalink',
    'cb_url',
    'facebook_url',
    'linkedin_url',
    'twitter_url',
    'logo_url',
    'domain',
    'state_code',
    'region',
    'city',
    'total_funding',
    'total_funding_currency_code',
    'rank',
    'type'  # just has type= 'organization' or 'person'
],axis=1)

In [72]:
investors_transform = distil_investors.copy()

In [73]:
# prepare top_500_investors for later to identify whether a firm is invested by top investors in a specific round
investors_sort = investors_transform.sort_values('investment_count',ascending=False) # top investors are chosen based on number of investments made. Higher investment count give a higher rank.
top_500_investors = investors_sort.head(500).reset_index()[['uuid','investment_count']] # after sorting, take the top 500 only
top_500_investors['is_top_500_investors'] = 1
top_500_investors = top_500_investors.rename(columns={'uuid':'investor_uuid'})

### 11. investments

In [74]:
distil_investments = investments.drop([
    'created_at',
    'updated_at',    
    'permalink',
    'cb_url',
    'type', # only 'investment'
    'rank',
    'investor_type' #only has 2 types, 'person' and 'organization'
],axis=1)

In [75]:
investments_transform = distil_investments.copy()

In [76]:
# using top_500_investors, identify i each investment entry is made by top investor (there could be multiple top investors in each funding round)
is_top_500_investors = investments_transform[['funding_round_uuid','investor_uuid']].merge(top_500_investors, how='left',on='investor_uuid')
is_top_500_investors = is_top_500_investors[['funding_round_uuid','is_top_500_investors']]
# count number of top investos in each funding round
num_top_500_investors = is_top_500_investors.groupby('funding_round_uuid').sum().rename(columns={'is_top_500_investors':'num_top_500_investors'})

# Part2  - Concatenating funding rounds related features

In [77]:
organizations_concat_p2 = organizations_p1.copy()

### acquisition
- -> acquired_on, has_acquirer, acquirer_name, is_acquisition_price

In [78]:
#prepare acquisitions for concat
acquisitions_concat = acquisitions_transform.copy()

# some companies were acquired for multiple times -> keep the earliest acquirer only for easier processing
acquisitions_concat = acquisitions_concat.sort_values('acquired_on').drop_duplicates('acquiree_uuid')

### funding_rounds
- investors -> num_top_500_investors in each round, number of investors each round

In [79]:
# prepare funding_rounds for concat
funding_rounds_concat = funding_rounds_transform.copy()

# rename uuid as funding_rounds_uuid
funding_rounds_concat = funding_rounds_concat.rename(columns={'uuid':'funding_round_uuid'})

# concat num_top_500_investors to funding_rounds.
funding_rounds_concat = funding_rounds_concat.merge(num_top_500_investors, how='left',on='funding_round_uuid')
# if null values -> no top investors -> fill with 0 
funding_rounds_concat['num_top_500_investors'] = funding_rounds_concat['num_top_500_investors'].fillna(0)

# create num top investors for each round
funding_rounds_concat['PreSeries_num_top_500_investors'] = funding_rounds_concat['is_PreSeries'] * funding_rounds_concat['num_top_500_investors']
funding_rounds_concat['RoundA_num_top_500_investors'] = funding_rounds_concat['is_RoundA'] * funding_rounds_concat['num_top_500_investors']
funding_rounds_concat['RoundB_num_top_500_investors'] = funding_rounds_concat['is_RoundB'] * funding_rounds_concat['num_top_500_investors']
funding_rounds_concat['RoundC_num_top_500_investors'] = funding_rounds_concat['is_RoundC'] * funding_rounds_concat['num_top_500_investors']
funding_rounds_concat['RoundD_num_top_500_investors'] = funding_rounds_concat['is_RoundD'] * funding_rounds_concat['num_top_500_investors']

# rename columns so they make sense later during concatenation
funding_rounds_concat = funding_rounds_concat.rename(columns={'org_uuid':'uuid','num_top_500_investors':'total_num_top_500_investors' })

# for each funding round with raised_amount positive -> we assume there must be at least one investor
funding_rounds_concat['investor_count']= funding_rounds_concat['investor_count'].fillna(1) 

In [80]:
# defin a helper function to help preparing features for each round - investors_count, announced_on, raised_amount_usd, post_money_valuation_usd
def prep_round_feature(name):
    funding_rounds_concat[name+'_investor_count']= np.where(funding_rounds_concat['is_'+name]==1, funding_rounds_concat['investor_count'],0)
    funding_rounds_concat[name+'_announced_on']= np.where(funding_rounds_concat['is_'+name]==1, funding_rounds_concat['announced_on'],0)
    funding_rounds_concat[name+'_raised_amount_usd']= np.where(funding_rounds_concat['is_'+name]==1, funding_rounds_concat['raised_amount_usd'],0)
    funding_rounds_concat[name+'_post_money_valuation_usd']= np.where(funding_rounds_concat['is_'+name]==1, funding_rounds_concat['post_money_valuation_usd'],0)

In [81]:
prep_round_feature('PreSeries')
prep_round_feature('RoundA')
prep_round_feature('RoundB')
prep_round_feature('RoundC')
prep_round_feature('RoundD')

In [82]:
# some companies have multiple rounds in one category (eg 2 times in roundC) -> we keep the date for the earliest round only
funding_rounds_dates = funding_rounds_concat[['uuid','PreSeries_announced_on','RoundA_announced_on','RoundB_announced_on','RoundC_announced_on','RoundD_announced_on']]
funding_rounds_dates = convert_datetime(funding_rounds_dates.replace(0, np.nan)) # 0 cannot be convert to datetime -> replace with NaN instead
funding_rounds_dates = funding_rounds_dates.groupby('uuid').min() # min takes the smallest dates only

# for companies have multiple rounds in one category (eg 2 times in roundC) -> features other than date would be the sum (eg. raised_amount = amount of first time + amount of second time)
funding_rounds_sort = funding_rounds_concat.groupby('uuid').sum()

# since sum() does not sum up dates, join the features+dates tgt
funding_rounds_concat = funding_rounds_dates.join(funding_rounds_sort)

### ipos
-  -> dates, price, etc.

In [83]:
# prepare ipos for concat
ipos_concat = ipos_transform.copy()
ipos_concat = convert_datetime(ipos_concat)

# keep the earlist ipos dates only 
ipos_dates = ipos_concat[['org_uuid','stock_symbol','went_public_on']]
ipos_dates = ipos_dates.sort_values('went_public_on').drop_duplicates('org_uuid') # sort by date then drop duplicate, so we keep the earliest one

# keep the largest prices only
ipos_price = ipos_concat[['org_uuid','share_price_usd','valuation_price_usd','money_raised_usd']]
ipos_price = ipos_price.groupby('org_uuid').max()

# merge the unique dates and unique prices -> unique ipos entry for each company
ipos_concat = ipos_dates.merge(ipos_price, how='left', on='org_uuid')
ipos_concat = ipos_concat.replace(0,np.nan) # 0 in fact means not available here

# boolean flags
ipos_concat['is_share_price_usd']= np.where(ipos_concat['share_price_usd'].notnull(),1,0) # 1 for share price, 0 for no
ipos_concat['is_valuation_price_usd']= np.where(ipos_concat['valuation_price_usd'].notnull(),1,0) # 1 for valuation, 0 for no
ipos_concat['is_money_raised_usd']= np.where(ipos_concat['money_raised_usd'].notnull(),1,0) # 1 for money raised, 0 for no

### org + acquisitions + funding_rounds + ipos

In [84]:
# concatenating acquisitions_concat along acquiree_uuid-->uuid

# rename acquiree_uuid as uuid
acquisitions_concat = acquisitions_concat.rename(columns= {"acquiree_uuid":"uuid"}) 

# concatenate
organizations_concat_p2= organizations_concat_p2.merge(acquisitions_concat,how='left', on='uuid')

# add boolean flags: has_acquirer and then fill missing values
organizations_concat_p2['has_acquirer']= np.where(organizations_concat_p2['acquirer_name'].notnull(),1,0)  
organizations_concat_p2['is_acquisition_price']= organizations_concat_p2['is_acquisition_price'].fillna(0)

In [85]:
# concatenating funding_rounds_concat along org_uuid-->uuid
                                                                                
# concatenate
organizations_concat_p2= organizations_concat_p2.merge(funding_rounds_concat,how='left', on='uuid')

In [86]:
#concatenating ipos_transform along org_uuid-->uuid

#rename acquiree_uuid as uuid
ipos_concat= ipos_concat.rename(columns= {"org_uuid":"uuid"})
                            
#creat copy and then concatenate
organizations_concat_p2 = organizations_concat_p2.merge(ipos_concat,how='left', on='uuid')

#fill missing values
organizations_concat_p2['is_share_price_usd']= organizations_concat_p2['is_share_price_usd'].fillna(0)
organizations_concat_p2['is_valuation_price_usd']= organizations_concat_p2['is_valuation_price_usd'].fillna(0)
organizations_concat_p2['is_money_raised_usd']= organizations_concat_p2['is_money_raised_usd'].fillna(0)

In [87]:
# organizations_concat_p2.to_csv('organizations_concat_p2.csv')

# Part3: Additional Columns

In [88]:
organizations_concat_p3 = organizations_concat_p2.copy()

### 1. Time taken between each round

In [89]:
# if the founded year is too old, the time between founded & certain round is too long and it gives error.
a = organizations_concat_p3[organizations_concat_p3['founded_on'].dt.year>1720] # so we calculate those after 1720 only
PreSeries_from_founded = a[a['PreSeries_announced_on'].notnull()]['PreSeries_announced_on'].dt.date - a[a['PreSeries_announced_on'].notnull()]['founded_on'].dt.date
PreSeries_from = pd.DataFrame({'PreSeries_from_founded': PreSeries_from_founded})
organizations_concat_p3 = pd.concat((organizations_concat_p3, PreSeries_from), axis = 1)
organizations_concat_p3 = organizations_concat_p3.reset_index(drop=True)

# googled, this should be a false record
organizations_concat_p3.loc[organizations_concat_p3['name']=='Arithmer', 'founded_on']=np.nan

In [90]:
# defining a helper function to calculate time between each round
def time_A_from_B(A, B):
    suffix2 = '_on' if B=='founded' else '_announced_on' 
    time = organizations_concat_p3[organizations_concat_p3[A+'_announced_on'].notnull()][A+'_announced_on'] - organizations_concat_p3[organizations_concat_p3[A+'_announced_on'].notnull()][B+suffix2]
    df = pd.DataFrame({A+'_from_'+B: time})
    new_org = pd.concat((organizations_concat_p3, df), axis = 1)
    new_org = new_org.reset_index(drop=True)
    return new_org

In [91]:
# combination of all periods
organizations_concat_p3 = time_A_from_B('RoundA','founded')
organizations_concat_p3 = time_A_from_B('RoundA','PreSeries')
organizations_concat_p3 = time_A_from_B('RoundB','founded')
organizations_concat_p3 = time_A_from_B('RoundB','PreSeries')
organizations_concat_p3 = time_A_from_B('RoundB','RoundA')
organizations_concat_p3 = time_A_from_B('RoundC','founded')
organizations_concat_p3 = time_A_from_B('RoundC','PreSeries')
organizations_concat_p3 = time_A_from_B('RoundC','RoundA')
organizations_concat_p3 = time_A_from_B('RoundC','RoundB')
organizations_concat_p3 = time_A_from_B('RoundD','founded')
organizations_concat_p3 = time_A_from_B('RoundD','PreSeries')
organizations_concat_p3 = time_A_from_B('RoundD','RoundA')
organizations_concat_p3 = time_A_from_B('RoundD','RoundB')
organizations_concat_p3 = time_A_from_B('RoundD','RoundC')

### 2. finding event_appearances between each round

In [92]:
# picking the essential columns for processing only
ea = event_appearances.drop(columns=['name','type','cb_url','created_at','updated_at','short_description','rank']).copy()

In [93]:
# Since there is no date column provvided, extract dates from event link, they look like XX-Conference-2012-2012329-XXX
ea['has_date'] = ea['permalink'].str.contains('-(\d{7,8})-',regex=True)
ea['date'] = ea['permalink'].str.extract('-(\d{7,8})-')
# some of them has year only
ea['has_year'] = ea['permalink'].str.contains('-(\d{4})-',regex=True)
ea['year'] = ea['permalink'].str.extract('-(\d{4})-')
# turn them to number/datetime
ea['year'] = pd.to_numeric(ea['year'])
ea['date'] = pd.to_datetime(ea['date'],format='%Y%m%d', yearfirst=True,errors='coerce')
ea.loc[ea['has_date'],'year'] = np.nan # remove years for the entries with a date

C:\Users\emily\Anaconda3\envs\TEMG4952A\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [94]:
# use the entries with date/year available only for counting, also drop useful col
ea_drop = ea[ea['has_date']|ea['has_year']].drop(columns=['uuid','permalink','event_uuid','event_name','appearance_type']).rename(columns={'participant_uuid':'uuid'}) #remove rows without any date/year
# split to two subsets: organization, person
ea_org = ea_drop[ea_drop['participant_type']=='organization']
ea_person = ea_drop[ea_drop['participant_type']=='person']

# concat dates for different rounds + event appearance with their corresponding date -> determine which period did the firm appear in event
ea_concat = organizations_concat_p3[['uuid','name','founded_on','closed_on','last_funding_on','PreSeries_announced_on','RoundA_announced_on','RoundB_announced_on','RoundC_announced_on','RoundD_announced_on']].copy()
ea_org = ea_org.merge(ea_concat, how='left', on='uuid')

# create boolean to identify each event -> if they happen before specific round
ea_org['PS_ea'] = ((ea_org.PreSeries_announced_on >= ea_org.date)|( ea_org.PreSeries_announced_on.dt.year>= ea_org.year))
ea_org['RA_ea'] = (ea_org.RoundA_announced_on >= ea_org.date)|( ea_org.RoundA_announced_on.dt.year>= ea_org.year)
ea_org['RB_ea'] = (ea_org.RoundB_announced_on >= ea_org.date)|( ea_org.RoundB_announced_on.dt.year>= ea_org.year)
ea_org['RC_ea'] = (ea_org.RoundC_announced_on >= ea_org.date)|( ea_org.RoundC_announced_on.dt.year>= ea_org.year)
ea_org['RD_ea'] = (ea_org.RoundD_announced_on >= ea_org.date)|( ea_org.RoundD_announced_on.dt.year>= ea_org.year)

# group by org uuid -> sum = count of how many times of event appearances
ea_sum = ea_org.groupby(by='uuid').sum().drop(['has_date','has_year','year'],axis=1)

# concatenate
organizations_concat_p3 = organizations_concat_p3.merge(ea_sum, how='left',on='uuid')

# fill NaN
organizations_concat_p3.loc[organizations_concat_p3.PreSeries_announced_on.isnull(), 'PS_ea'] = np.nan
organizations_concat_p3.loc[organizations_concat_p3.RoundA_announced_on.isnull(), 'RA_ea'] = np.nan
organizations_concat_p3.loc[organizations_concat_p3.RoundB_announced_on.isnull(), 'RB_ea'] = np.nan
organizations_concat_p3.loc[organizations_concat_p3.RoundC_announced_on.isnull(), 'RC_ea'] = np.nan
organizations_concat_p3.loc[organizations_concat_p3.RoundD_announced_on.isnull(), 'RD_ea'] = np.nan

# rename the counts to proper names
organizations_concat_p3 = organizations_concat_p3.rename(columns={'PS_ea':'PreSeries_num_ea_org',
                                                                               'RA_ea':'RoundA_num_ea_org',
                                                                               'RB_ea':'RoundB_num_ea_org',
                                                                               'RC_ea':'RoundC_num_ea_org',
                                                                               'RD_ea':'RoundD_num_ea_org'})

In [95]:
organizations_concat_p3.info(verbose=True)
# organizations_concat_p3.to_csv('439k_dataset.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439847 entries, 0 to 439846
Data columns (total 124 columns):
 #    Column                                       Dtype          
---   ------                                       -----          
 0    uuid                                         object         
 1    name                                         object         
 2    cb_url                                       object         
 3    domain                                       object         
 4    homepage_url                                 object         
 5    country_code                                 object         
 6    state_code                                   object         
 7    category_list                                object         
 8    category_groups_list                         object         
 9    num_funding_rounds                           float64        
 10   total_funding_usd                            float64        
 11   founded_on 

# Part4 VGR calculation

### 1. dataset selction (for modelling)

In [96]:
dataset = organizations_concat_p3.copy()
dataset = dataset.drop([
    'category_list',
    'founded_on',
    'last_funding_on',
    'closed_on',
    'acquired_on',
    'age_operating',
    'age_closed',
    'founded_on_year',
    'closed_on_year',
    'parent_uuid',
    'parent_name',
    'employee_count', #these features are time-dependent and could not be used to predict (they are useful for identifying existing big companies only)
], axis=1)

# only choose those are primarily company. Others are investors/schools that UBS is not interested in
dataset = dataset[dataset['is_primary_company'] == 1]

In [97]:
# Only use companies within UBS's interested categories
categories_of_interest = [
    'Financial Services',
    'Lending and Investments',
    'Payments',
    'Artificial Intelligence',
    'Data and Analytics',
    'Platforms',
    'Privacy and Security',
]

# get the categories a company belongs to 
dataset['category_groups_list'] = dataset['category_groups_list'].apply(lambda x: strip_uninterested_categories(x, categories_of_interest))
category_groups_list = dataset['category_groups_list'].fillna('').apply(split_categories)
categories_of_interest_getter = category_groups_list.apply(lambda x: any(y in x for y in categories_of_interest))
dataset = dataset[categories_of_interest_getter]

# one-hot encode the category the company belongs to
category_map = dataset.category_groups_list.str.split(',', expand=True).stack()
category_dummies = pd.get_dummies(category_map, prefix='is_category').groupby(level=0).sum()
dataset = pd.concat((dataset, category_dummies), axis = 1).drop(['category_groups_list'], axis=1)

### 2.  fill in missing valuation

In [98]:
'''
We evaluate a company's success by Valuation Growth Rate (VGR)
VGR from t0 to t1 = (valuation at t1 / valuation at t0) ^ (1 / (t1-t0)) -1

Given the sparse data in valuation, we fill in them by using raised_amount_usd

1.  we calculate to average equity ratio from companies that has both raised_amount and post_valuation 
by get_equity_ratio() for each round.

2.  Then if the post_valuation is empty, we fill it with (raised_amount / avg equity ratio)
'''
def get_equity_ratio(df, raised_amount_col, post_money_col):
    funding_data = dataset[[raised_amount_col,post_money_col]]
    funding_data[raised_amount_col] = funding_data[raised_amount_col].replace(0,np.nan)
    funding_data[post_money_col] = funding_data[post_money_col].replace(0,np.nan)
    funding_data = funding_data[funding_data[raised_amount_col].notnull() & funding_data[post_money_col].notnull()]

    funding_data['equity_ratio'] = funding_data[raised_amount_col] / funding_data[post_money_col]
    return funding_data['equity_ratio'].mean()

In [99]:
avg_preSeries_equity_ratio = get_equity_ratio(
    dataset,
    'PreSeries_raised_amount_usd',
    'PreSeries_post_money_valuation_usd'
)
avg_roundA_equity_ratio = get_equity_ratio(
    dataset,
    'RoundA_raised_amount_usd',
    'RoundA_post_money_valuation_usd'
)
avg_roundB_equity_ratio = get_equity_ratio(
    dataset,
    'RoundB_raised_amount_usd',
    'RoundB_post_money_valuation_usd'
)
avg_roundC_equity_ratio = get_equity_ratio(
    dataset,
    'RoundC_raised_amount_usd',
    'RoundC_post_money_valuation_usd'
)
avg_roundD_equity_ratio = get_equity_ratio(
    dataset,
    'RoundD_raised_amount_usd',
    'RoundD_post_money_valuation_usd'
)

C:\Users\emily\Anaconda3\envs\TEMG4952A\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\emily\Anaconda3\envs\TEMG4952A\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [100]:
avg_preSeries_equity_ratio

0.28094025523667615

In [101]:
avg_roundA_equity_ratio

0.2755640808542754

In [102]:
avg_roundB_equity_ratio

0.2262574455112157

In [103]:
avg_roundC_equity_ratio

0.18094789325964744

In [104]:
avg_roundD_equity_ratio

0.2127669927118955

In [105]:
# step 2 as mentioned about, fill in valuation

def fill_valuation(x1,x2,factor):
    x2 = x1 / factor if (x2 is np.nan or x2 == 0) else x2
    return x2

In [106]:
dataset['PreSeries_post_money_valuation_usd_augmented'] = dataset.apply(lambda x: fill_valuation(
    x['PreSeries_raised_amount_usd'],
    x['PreSeries_post_money_valuation_usd'],
    avg_preSeries_equity_ratio),axis=1)
dataset['RoundA_post_money_valuation_usd_augmented'] = dataset.apply(lambda x: fill_valuation(
    x['RoundA_raised_amount_usd'],
    x['RoundA_post_money_valuation_usd'],
    avg_roundA_equity_ratio),axis=1)
dataset['RoundB_post_money_valuation_usd_augmented'] = dataset.apply(lambda x: fill_valuation(
    x['RoundB_raised_amount_usd'],
    x['RoundB_post_money_valuation_usd'],
    avg_roundB_equity_ratio),axis=1)
dataset['RoundC_post_money_valuation_usd_augmented'] = dataset.apply(lambda x: fill_valuation(
    x['RoundC_raised_amount_usd'],
    x['RoundC_post_money_valuation_usd'],
    avg_roundC_equity_ratio),axis=1)
dataset['RoundD_post_money_valuation_usd_augmented'] = dataset.apply(lambda x: fill_valuation(
    x['RoundD_raised_amount_usd'],
    x['RoundD_post_money_valuation_usd'],
    avg_roundD_equity_ratio),axis=1)

### 3. calculate VGR, drop null entries

VGR from t0 to t1 = (valuation at t1 / valuation at t0) ^ (1 / (t1-t0)) -1

In [107]:
# 'ROI' refers to VGR
# get_ROI calculates VGR of all time (if data available)
def get_ROI(pre_v, A_v, B_v, C_v, D_v, pre_a, A_a, B_a, C_a, D_a):
        time_naught = None
        time_prime = None
        valuation_naught = None
        valuation_prime = None
        valuations = [pre_v, A_v, B_v, C_v, D_v]
        dates = [pre_a, A_a, B_a, C_a, D_a]
        for i in range(5):
            if valuations[i] and dates[i]:
                time_naught = dates[i]
                valuation_naught = valuations[i]
                break
        for i in range(4,-1,-1):
            if valuations[i] and dates[i]:
                time_prime = dates[i]
                valuation_prime = valuations[i]
                break
        if not (time_naught and time_prime) or (time_naught == time_prime):
            return np.nan
        num_years = (time_prime - time_naught).days/365
        try:
            roi = (valuation_prime / valuation_naught) ** (1/num_years) - 1
            if not roi.imag == 0:
                return np.nan
        except:
            return np.nan
        return roi.real # only return real number

In [108]:
# 'ROI' refers to VGR
# get_ROI_from_to calculates VGR from t0 to t1

def get_ROI_from_to(from_v, to_v, from_a, to_a):
    time_naught = from_a
    time_prime = to_a
    valuation_naught = from_v
    valuation_prime = to_v
    if not (time_naught and time_prime and valuation_naught and valuation_prime) or (time_naught == time_prime):
        return np.nan
    num_years = (time_prime - time_naught).days/365
    try:
        roi = (valuation_prime / valuation_naught) ** (1/num_years) - 1
        if not roi.imag == 0:
            return np.nan
    except:
        return np.nan
    return roi.real # only return real number
    

In [109]:
# make sure all dates are in timestamp format
dataset = convert_datetime(dataset)

In [110]:
# All time ROI (VGR)
dataset['all_time_roi'] = dataset.apply(lambda x: get_ROI(
    x['PreSeries_post_money_valuation_usd_augmented'],
    x['RoundA_post_money_valuation_usd_augmented'],
    x['RoundB_post_money_valuation_usd_augmented'],
    x['RoundC_post_money_valuation_usd_augmented'],
    x['RoundD_post_money_valuation_usd_augmented'],
    x['PreSeries_announced_on'],
    x['RoundA_announced_on'],
    x['RoundB_announced_on'],
    x['RoundC_announced_on'],
    x['RoundD_announced_on']
),axis=1)

dataset = dataset[dataset['all_time_roi'].notnull()]

In [111]:
# VGR from PreSeries to RoundA
dataset['roi_from_PS_to_RA'] = dataset.apply(lambda x: get_ROI_from_to(
    x['PreSeries_post_money_valuation_usd_augmented'],
    x['RoundA_post_money_valuation_usd_augmented'],
    x['PreSeries_announced_on'],
    x['RoundA_announced_on'],
),axis=1)
dataset['roi_from_PS_to_RA'].notnull().sum()

3827

In [112]:
# VGR from PreSeries to RoundB
dataset['roi_from_PS_to_RB'] = dataset.apply(lambda x: get_ROI_from_to(
    x['PreSeries_post_money_valuation_usd_augmented'],
    x['RoundB_post_money_valuation_usd_augmented'],
    x['PreSeries_announced_on'],
    x['RoundB_announced_on'],
),axis=1)
dataset['roi_from_PS_to_RB'].notnull().sum()

1654

In [113]:
# VGR from PreSeries to RoundC
dataset['roi_from_PS_to_RC'] = dataset.apply(lambda x: get_ROI_from_to(
    x['PreSeries_post_money_valuation_usd_augmented'],
    x['RoundC_post_money_valuation_usd_augmented'],
    x['PreSeries_announced_on'],
    x['RoundC_announced_on'],
),axis=1)
dataset['roi_from_PS_to_RC'].notnull().sum()

659

In [114]:
# VGR from PreSeries to RoundD
dataset['roi_from_PS_to_RD'] = dataset.apply(lambda x: get_ROI_from_to(
    x['PreSeries_post_money_valuation_usd_augmented'],
    x['RoundD_post_money_valuation_usd_augmented'],
    x['PreSeries_announced_on'],
    x['RoundD_announced_on'],
),axis=1)
dataset['roi_from_PS_to_RD'].notnull().sum()

2953

In [115]:
# VGR from PreSeries to IPO
dataset['roi_from_PS_to_IPO'] = dataset.apply(lambda x: get_ROI_from_to(
    x['PreSeries_post_money_valuation_usd_augmented'],
    x['valuation_price_usd'],
    x['PreSeries_announced_on'],
    x['went_public_on'],
),axis=1)
dataset['roi_from_PS_to_IPO'].notnull().sum()

47

In [116]:
# VGR from RoundA to RoundB
dataset['roi_from_RA_to_RB'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundA_post_money_valuation_usd_augmented'],
    x['RoundB_post_money_valuation_usd_augmented'],
    x['RoundA_announced_on'],
    x['RoundB_announced_on'],
),axis=1)
dataset['roi_from_RA_to_RB'].notnull().sum()

3408

In [117]:
# VGR from RoundA to RoundC
dataset['roi_from_RA_to_RC'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundA_post_money_valuation_usd_augmented'],
    x['RoundC_post_money_valuation_usd_augmented'],
    x['RoundA_announced_on'],
    x['RoundC_announced_on'],
),axis=1)
dataset['roi_from_RA_to_RC'].notnull().sum()

1540

In [118]:
# VGR from RoundA to RoundD
dataset['roi_from_RA_to_RD'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundA_post_money_valuation_usd_augmented'],
    x['RoundD_post_money_valuation_usd_augmented'],
    x['RoundA_announced_on'],
    x['RoundD_announced_on'],
),axis=1)
dataset['roi_from_RA_to_RD'].notnull().sum()

2310

In [119]:
# VGR from RoundA to IPO
dataset['roi_from_RA_to_IPO'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundA_post_money_valuation_usd_augmented'],
    x['valuation_price_usd'],
    x['RoundA_announced_on'],
    x['went_public_on'],
),axis=1)
dataset['roi_from_RA_to_IPO'].notnull().sum()

121

In [120]:
# VGR from RoundB to RoundC
dataset['roi_from_RB_to_RC'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundB_post_money_valuation_usd_augmented'],
    x['RoundC_post_money_valuation_usd_augmented'],
    x['RoundB_announced_on'],
    x['RoundC_announced_on'],
),axis=1)
dataset['roi_from_RB_to_RC'].notnull().sum()

1770

In [121]:
# VGR from RoundB to RoundD
dataset['roi_from_RB_to_RD'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundB_post_money_valuation_usd_augmented'],
    x['RoundD_post_money_valuation_usd_augmented'],
    x['RoundB_announced_on'],
    x['RoundD_announced_on'],
),axis=1)
dataset['roi_from_RB_to_RD'].notnull().sum()

1728

In [122]:
# VGR from RoundB to IPO
dataset['roi_from_RB_to_IPO'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundB_post_money_valuation_usd_augmented'],
    x['valuation_price_usd'],
    x['RoundB_announced_on'],
    x['went_public_on'],
),axis=1)
dataset['roi_from_RB_to_IPO'].notnull().sum()

119

In [123]:
# VGR from RoundC to RoundD
dataset['roi_from_RC_to_RD'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundC_post_money_valuation_usd_augmented'],
    x['RoundD_post_money_valuation_usd_augmented'],
    x['RoundC_announced_on'],
    x['RoundD_announced_on'],
),axis=1)
dataset['roi_from_RC_to_RD'].notnull().sum()

1191

In [124]:
# VGR from RoundC to IPO
dataset['roi_from_RC_to_IPO'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundC_post_money_valuation_usd_augmented'],
    x['valuation_price_usd'],
    x['RoundC_announced_on'],
    x['went_public_on'],
),axis=1)
dataset['roi_from_RC_to_IPO'].notnull().sum()

99

In [125]:
# VGR from RoundD to IPO
dataset['roi_from_RD_to_IPO'] = dataset.apply(lambda x: get_ROI_from_to(
    x['RoundD_post_money_valuation_usd_augmented'],
    x['valuation_price_usd'],
    x['RoundD_announced_on'],
    x['went_public_on'],
),axis=1)
dataset['roi_from_RD_to_IPO'].notnull().sum()

111

In [126]:
dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8865 entries, 26 to 439282
Data columns (total 139 columns):
 #    Column                                        Dtype          
---   ------                                        -----          
 0    uuid                                          object         
 1    name                                          object         
 2    cb_url                                        object         
 3    domain                                        object         
 4    homepage_url                                  object         
 5    country_code                                  object         
 6    state_code                                    object         
 7    num_funding_rounds                            float64        
 8    total_funding_usd                             float64        
 9    facebook_url                                  object         
 10   linkedin_url                                  object         
 11  

In [127]:
# dataset.to_csv('dataset.csv',index=False)

# Part5 Binning

In [128]:
NUM_LABEL = 5
lb_make = LabelEncoder()

In [129]:
bin_data = dataset.copy()

In [130]:
bin_data = convert_datetime(bin_data)

In [131]:
# These columns are dropped for the modelling dataset. Commented so they don't give error

# bin_data['founded_on_binned'] = pd.qcut(bin_data['founded_on'], q=NUM_LABEL,precision = 0)
# bin_data['last_funding_on_binned'] = pd.qcut(bin_data['last_funding_on'], q=NUM_LABEL,precision = 0)
# bin_data['closed_on_binned'] = pd.qcut(bin_data['closed_on'], q=NUM_LABEL,precision = 0)
# bin_data['age_closed_binned'] = pd.qcut(bin_data['age_closed'], q=NUM_LABEL,precision = 0)
# bin_data['age_operating_binned'] = pd.qcut(bin_data['age_operating'], q=NUM_LABEL,precision = 0)
# bin_data['acquired_on_binned'] = pd.qcut(bin_data['acquired_on'], q=NUM_LABEL,precision = 0)

# for this version, null values -> biggest bin (bin 5)
# bin_data['founded_on_binned'] = lb_make.fit_transform(bin_data['founded_on_binned'])
# bin_data['last_funding_on_binned'] = lb_make.fit_transform(bin_data['last_funding_on_binned'])
# bin_data['closed_on_binned'] = lb_make.fit_transform(bin_data['closed_on_binned'])
# bin_data['age_closed_binned'] = lb_make.fit_transform(bin_data['age_closed_binned'])
# bin_data['age_operating_binned'] = lb_make.fit_transform(bin_data['age_operating_binned'])
# bin_data['acquired_on_binned'] = lb_make.fit_transform(bin_data['acquired_on_binned'])

In [132]:
# Bin the time taken for each round for classification
bin_data['PreSeries_from_founded_binned'] = pd.qcut(bin_data['PreSeries_from_founded'], q=NUM_LABEL,precision = 0)
bin_data['RoundA_from_founded_binned'] = pd.qcut(bin_data['RoundA_from_founded'], q=NUM_LABEL,precision = 0)
bin_data['RoundB_from_founded_binned'] = pd.qcut(bin_data['RoundB_from_founded'], q=NUM_LABEL,precision = 0)
bin_data['RoundC_from_founded_binned'] = pd.qcut(bin_data['RoundC_from_founded'], q=NUM_LABEL,precision = 0)
bin_data['RoundD_from_founded_binned'] = pd.qcut(bin_data['RoundD_from_founded'], q=NUM_LABEL,precision = 0)

In [133]:
bin_data['PreSeries_from_founded_binned'] = lb_make.fit_transform(bin_data['PreSeries_from_founded_binned'])
bin_data['RoundA_from_founded_binned'] = lb_make.fit_transform(bin_data['RoundA_from_founded_binned'])
bin_data['RoundB_from_founded_binned'] = lb_make.fit_transform(bin_data['RoundB_from_founded_binned'])
bin_data['RoundC_from_founded_binned'] = lb_make.fit_transform(bin_data['RoundC_from_founded_binned'])
bin_data['RoundD_from_founded_binned'] = lb_make.fit_transform(bin_data['RoundD_from_founded_binned'])

In [134]:
# bin_data.to_csv('dataset_bineed.csv',index=False)

# Part6 - Additional processing for front-end display : Continent names, make strings to number

### 1. Continent names

In [135]:
dataset_p6 = bin_data.copy()

In [136]:
# Get unique country codes and their corresponding frequency
countries = dataset_p6.country_code.value_counts()
countries = pd.DataFrame(data={'country_code':countries.index,'frequency':countries.values})

In [137]:
# initialze a list to store the unknown country codes/ regions that we want to put to the 'others' category
others=[]

In [138]:
# For each country code, get it's country object
for index, row in countries.iterrows():
    try:
        countries.loc[index, 'object'] = pycountry.countries.get(alpha_3=row['country_code'])
    except:
        others.append(row['country_code'])
        print('failed for', row['country_code'])
        
# For each country object, get it's country's continent code
for index, row in countries.iterrows():
    try:
        countries.loc[index, 'continent'] = pc.country_alpha2_to_continent_code(row['object'].alpha_2)
    except:
        others.append(row['country_code'])
        print('failed for', row['country_code'])
        
# For each country continent code, get it's country's continent name       
for index, row in countries.iterrows():
    try:
        countries.loc[index, 'continent_name'] = pc.convert_continent_code_to_continent_name(row['continent'])
    except:
        others.append(row['country_code'])
        print('failed for', row['country_code'])

failed for Unknown
failed for ROM
failed for TAN
failed for Unknown
failed for ROM
failed for TAN


In [139]:
# Since number of companies in these two continents is small -> put them in Others category
others.extend(['Oceania','Africa'])

In [140]:
# since the library put 'USA' in continent 'North America', but the number of companies in USA is huge 
# -> separate a category for USA, other countries from NA and SA are put into 'Americas'
for index, row in countries.iterrows():
    if row['country_code']=='USA':
        countries.loc[index, 'final_continent_name'] = row['country_code']
    elif row['country_code'] in others or row['continent_name'] in others:
        countries.loc[index, 'final_continent_name'] = 'Others'
    elif row['continent'] in ['NA','SA']:
        countries.loc[index, 'final_continent_name'] = 'Americas'
    else:
        countries.loc[index, 'final_continent_name'] = row['continent_name']

In [141]:
# display how many companies belongs to each category
countries.groupby('final_continent_name').sum()

,frequency
final_continent_name,
Americas,385
Asia,1235
Europe,1640
Others,157
USA,5448


In [142]:
dataset_p6 = dataset_p6.merge(countries[['country_code','final_continent_name']],on='country_code')
dataset_p6 = dataset_p6.rename(columns={'final_continent_name':'country_continent_name'})

### 2. Remove 'days' from certain columns

In [143]:
# These entries are in timedelta format / string format
days = ['PreSeries_from_founded',
        'RoundA_from_founded',
 'RoundA_from_PreSeries',
 'RoundB_from_RoundA',
 'RoundB_from_PreSeries',
 'RoundB_from_founded',
 'RoundC_from_RoundB',
 'RoundC_from_RoundA',
 'RoundC_from_PreSeries',
 'RoundC_from_founded',
 'RoundD_from_RoundC',
 'RoundD_from_RoundB',
 'RoundD_from_RoundA',
 'RoundD_from_PreSeries',
 'RoundD_from_founded']

In [144]:
# make them into number which directly represent how many days
for day in days:
    dataset_p6[day] = pd.to_timedelta(dataset_p6[day]).dt.days

In [ ]:
# output and save the final dataset
dataset_p6.to_csv(output_path+'dataset.csv',index=False)